In [1]:
import scanpy as sc
import os
import numpy as np
from tqdm.notebook import tqdm
import glob
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import scipy.stats as stats

##### Add the paths to the MERSCOPE datasets.

In [2]:
input_folders = glob.glob('/projects/2023_Spatial_Paper/Analysis_Alex/merscope_final/SI*') 

##### Add the path to the cell-typed object

In [3]:
output_folder = r'/projects/2023_Spatial_Paper/Analysis_Alex/merscope_final/analysis/final'
combined_adata = sc.read(os.path.join(output_folder, 'full_celltypes_and_leiden.h5ad'))

/home/amonell/.local/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


##### Subset the axes-calculated objects to only include cells from the full celltyped object

In [ ]:
for input_file in input_folders:
    for roll in ['roll1', 'roll2']:
        adata = sc.read(os.path.join(input_file, 'adatas', f'07_axes_defined_{roll}.h5ad'))
        
        sub_combined = combined_adata[combined_adata.obs['batch'] == os.path.basename(input_file)+ f'_roll_{roll}']
        sub_combined.obs['indices'] = [i for i in range(len(sub_combined.obs.index))]
        indices = adata.obs.merge(sub_combined.obs, left_on='cell', right_index=True, how='left')['indices'].values
        adata = adata[np.where(np.invert(np.isnan(indices)))[0], :]
        
        indices = indices[np.where(np.invert(np.isnan(indices)))]
        indices = indices.astype(int)
        adata.obsm['X_mde'] = sub_combined.obsm['X_mde'][indices, :]
        adata.obsm['X_scVI'] = sub_combined.obsm['X_scVI'][indices, :]
        if 'predicted_longitudinal' not in adata.obs.columns:
            adata.obs['predicted_longitudinal'] = adata.obs['longitudinal']
        adata.obs['batch'] = os.path.basename(input_file)+ f'_roll_{roll}'
        adata.write(os.path.join(input_file, 'adatas', f'08_final_adata_{roll}.h5ad'))

#### Concatenate all rolls

In [ ]:
adatalist = []
for input_file in input_folders:
    for roll in ['roll1', 'roll2']:
        adatalist.append(sc.read(os.path.join(input_file, 'adatas', f'08_final_adata_{roll}.h5ad')))
concatenated_all = sc.concat(adatalist)

concatenated_all.obs['epithelial_distance_clipped'] = np.clip(concatenated_all.obs['epithelial_distance'].values, 0, 0.6)


##### Flip all longitudinal axes into the same orientation based on polarization of Guca2a

In [7]:
unique_bat = np.unique(concatenated_all.obs.batch)

In [41]:

sc.set_figure_params(dpi=150)
outer_inner = []
for i in unique_bat:
    longit = concatenated_all[(concatenated_all.obs.batch == i) & concatenated_all.obs.not_removed_from_longitudinal].obs['predicted_longitudinal']
    guca2a = concatenated_all[(concatenated_all.obs.batch == i) & concatenated_all.obs.not_removed_from_longitudinal].X[:, concatenated_all[concatenated_all.obs.batch == i].var.index.isin(['Fdft1'])].flatten()
    spear = stats.spearmanr(longit, guca2a)
    if spear.correlation > 0:
        outer_inner.append('inner')
    else:
        outer_inner.append('outer')
    #sc.pl.embedding(concatenated_all[concatenated_all.obs.batch == i], basis='spatial', color='Guca2a', vmax=1, cmap='Blues', title=i)

In [42]:
longitude = {}
for bt in range(len(unique_bat)):
    longitude[unique_bat[bt]] = outer_inner[bt]

In [44]:
guca2a_expression = [longitude.get(i) for i in concatenated_all.obs.batch.values]
new_longitudinal = np.zeros(len(concatenated_all.obs))

for i in unique_bat:
    indices = np.where(concatenated_all.obs.batch.values == i)[0]
    if longitude.get(i) == 'outer':
        new_longitudinal[indices] = 1 - concatenated_all[indices, :].obs.predicted_longitudinal.values
    elif longitude.get(i) == 'inner':
        new_longitudinal[indices] = concatenated_all[indices, :].obs.predicted_longitudinal.values

concatenated_all.obs['predicted_longitudinal'] = new_longitudinal

##### Write out the final MERSCOPE object

In [46]:
concatenated_all.write(os.path.join(output_folder, 'final_celltyped_and_axes.h5ad'))